# ANOVA

## Carga y lectura de datos de una regresión con un tratamiento

In [ ]:
!wget -O rehab.csv https://raw.githubusercontent.com/jordipereiragude/dataforcourses/refs/heads/main/rehab.csv

In [ ]:
df=pd.read_csv("rehab.csv")
print(df.to_markdown())

Representación gráfica

In [ ]:
import plotly.express as px

# Crear el diagrama de cajas y bigotes
fig = px.box(df, x='Fitness', y='Time',
             title='Distribución del Tiempo según el Nivel de Fitness',
             labels={'Fitness': 'Nivel de Fitness', 'Time': 'Tiempo'})

# Mostrar el gráfico
fig.show()

Vamos a construir un modelo ANOVA con un factor. Veremos los resultados tanto en el formato típico de ANOVA como de regresión

In [ ]:
import statsmodels.formula.api as smf
import statsmodels.api as sm

# Crear un modelo de regresión lineal para el ANOVA
# C(Fitness) indica que Fitness debe ser tratado como una variable categórica
model_anova = smf.ols('Time ~ C(Fitness)', data=df).fit()

# Realizar el test ANOVA
anova_table = sm.stats.anova_lm(model_anova, typ=2)

# Imprimir la tabla ANOVA
print("Tabla ANOVA:")
print(anova_table.to_markdown())

In [ ]:
print(model_anova.summary())

Tras hacer el test que nos indica que el tratamiento, en este caso Fitness, sí tiene efecto en la variable respuesta.

Todavía es necesario identificar qué impacto tiene cada tratamiento. Para ello realizamos un test de Tukey.

In [ ]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Realizar el test de Tukey HSD
tukey_result = pairwise_tukeyhsd(endog=df['Time'], groups=df['Fitness'], alpha=0.05)

# Imprimir el resumen de los resultados del test de Tukey
print("Resultados del Test de Tukey HSD:")
print(tukey_result)

También podemos visualizar estas diferencias a través de un gráfico que compara los intervalos que el modelo cree que tiene el efecto de cada variable

In [ ]:
import matplotlib.pyplot as plt

# Obtener el resultado del test de Tukey (asumiendo que tukey_result ya está definido)
# Si no lo estuviera, se necesitaría ejecutar la celda anterior para obtenerlo.
# tukey_result = pairwise_tukeyhsd(endog=df['Time'], groups=df['Fitness'], alpha=0.05)

# Crear la representación visual del test de Tukey
tukey_result.plot_simultaneous(ylabel='Nivel de Fitness', xlabel='Tiempo Promedio')
plt.title('Comparación Simultánea de Medias - Test de Tukey HSD')
plt.show()

Un modelo ANOVA no deja de ser una regresión lineal, así que debemos verificar la normalidad de los residuos

In [ ]:
import statsmodels.api as sm
import matplotlib.pyplot as plt

# Obtener los residuos del modelo_anova
residuals_anova = model_anova.resid

# Crear el Q-Q plot
sm.qqplot(residuals_anova, line='s')
plt.title('Q-Q Plot de los Residuos del Modelo ANOVA')
plt.xlabel('Cuantiles Teóricos')
plt.ylabel('Cuantiles de los Residuos')
plt.show()

En caso que los residuos no sean normales todavía es posible recurrir a modelos que no requieran normalidad. En este caso podemos usar una prueba como Kurskal-Wallis

In [ ]:
from scipy import stats

# Separar los datos de 'Time' por cada nivel de 'Fitness'
fitness_groups = [df['Time'][df['Fitness'] == level] for level in df['Fitness'].unique()]

# Realizar el test de Kruskal-Wallis
h_statistic, p_value = stats.kruskal(*fitness_groups)

print(f"Resultados del Test de Kruskal-Wallis:")
print(f"H-statistic: {h_statistic:.4f}")
print(f"P-value: {p_value:.4f}")

# Interpretar el resultado
alpha = 0.05
if p_value < alpha:
    print(f"Con un p-value de {p_value:.4f} (menor que {alpha}), rechazamos la hipótesis nula.")
    print("Conclusión: Existen diferencias estadísticamente significativas en la mediana del Time entre al menos dos grupos de Fitness.")
else:
    print(f"Con un p-value de {p_value:.4f} (mayor que {alpha}), no rechazamos la hipótesis nula.")
    print("Conclusión: No hay evidencia suficiente para afirmar diferencias significativas en la mediana del Time entre los grupos de Fitness.")